# Notebook for running MSRE calculations directly from a CAD drawing
This notebook show an example of running computations of a model of the Moten Salt Reactor Experiment of MSRE in short.
The model itself has been generated from tehe original drawings from Oak Ridge National lab 

The CAD-model is available on github at https://github.com/openmsr/msre, which in turn is generated from a long list of documents which have been compiled at https://github.com/openmsr/msr-archive

The simulation backend is run using the Open Source Monte Carlo particle transport code OpenMC (https://openmc.org), through its' python interface.

**Important: If you want your work to be available after you shutdown the docker, you must copy your notebooks to a location mounted on your local machine.**

If you started the docker using the supplied ```run_docker.sh```-script, the ```notebooks```-directory has been mounted like this.

## The (obvious) 1st step is to import the OpenMC python interface

In [ ]:
import openmc

Next we define a set of materials objects that form the core of the MSRE, graphite,  hastelloy N / inor-8, inconel, the fuel salt, and helium. Lastly these are exported to am OpenMC-xml control file.

In [ ]:
graphite=openmc.Material(name='graphite')
graphite.add_element('C',1.0,'ao')
graphite.set_density('g/cc',2.26)

#Hastelloy N / INOR-8 nominal material composition from
#ORNL-TM-4189
inor=openmc.Material(name='inor')
inor.add_element('Ni',0.72)
inor.add_element('Mo',0.16)
inor.add_element('Cr',0.07)
inor.add_element('Fe',0.05)
inor.set_density('g/cc',9)

# LiF,BeF2,UF4,ZrF4 [0.67,0.23,0.05,0.0079] mol % @ 33% enrichment 
molar_comp={'LiF':0.67,'BeF2':0.23, 'ZrF4':0.05, 'UF4':0.0079}
enrichment=0.5
salt=openmc.Material(name='salt')
salt.add_element('F',molar_comp['LiF']*1/2+molar_comp['BeF2']*2/3+molar_comp['ZrF4']*4/5+molar_comp['UF4']*4/5,'ao')
salt.add_nuclide('Li7',molar_comp['LiF']*1/2,'ao')
salt.add_element('Be',molar_comp['BeF2']*1/3,'ao')
salt.add_element('Zr',molar_comp['ZrF4']*1/5,'ao')
salt.add_nuclide('U235',enrichment*molar_comp['UF4']*1/5,'ao')
salt.add_nuclide('U238',(1-enrichment)*molar_comp['UF4']*1/5,'ao')
salt.set_density('g/cc',2.2)

# The natural isotopes have been used for this alloy
# The density is set to that of Ni.
inconel=openmc.Material(name='inconel')
inconel.add_element('Ni',0.72,'ao')
inconel.add_element('Cr',0.20,'ao')
inconel.add_element('Fe',0.08,'ao')
inconel.set_density('g/cc',8.9)

helium=openmc.Material(name='helium')
helium.add_nuclide('He4',1.0,'ao')
helium.set_density('g/cc',1.0e-4)

materials=openmc.Materials([helium_comp,salt,salt2,graphite,inconel,inor])
materials.export_to_xml()

As a control we can inspect the materials object. Notice how OpenMC has in the revant cases expanded our material definition to consist the naturally occurring  isotope concentrations.

In [ ]:
materials

In [ ]:
#geometry
h5m_filepath="dagmc.h5m"
dag_univ = openmc.DAGMCUniverse(h5m_filepath)
geom = openmc.Geometry(root=dag_univ)
geom.export_to_xml()

We can now plot our geometry to verify that this is in fact the geometry we want. We plot two slices (xz and xy) through the centre of the MSRE core, and color the geometry by constituent material.

In [ ]:
xwidth = 350
yheight = 350
material_colors={salt:'red', inor:'lightblue', inconel:'blue',helium:'white',graphite:'gray'}
#xz plot
p1 = openmc.Plot()
p1.background='white'
p1.basis = 'xz'
p1.width = (xwidth,yheight)
p1.origin=(0,0,125)
p1.pixels = (800, 800)
p1.color_by = 'material'
p1.colors=material_colors
#xy plot
p2 = openmc.Plot()
p2.background='white'
p2.basis='xy'
p2.width=(xwidth, yheight)
p2.pixels = (800,800)
p2.origin=(0,0,100)
p2.color_by='material'
p2.colors=material_colors

plots=openmc.Plots([p1,p2])
openmc.plot_inline(plots)

Now we need to define som settings for our calculations.

First of all - we need to some neutrons to kick-start or chain reaction. In OpenMC this is doen by defining a source region. Here this is simply defined as being a region that encloses the MSRE core.

Next we define some settings for the Monte Carlo-computation, such as how many particles we would initially run with. 

After the members of the settings python object have been filled to our desires, we export this to a settings xml-file

In [ ]:
# Create a neutron source for kick-starting
source_volume=openmc.stats.Box([-125,-125,0],[125,125,500], only_fissionable=True)
source = openmc.Source(space=source_volume)
source.angle=openmc.stats.Isotropic()

In [ ]:
#Finally we build a settings object for OpenMC where we define parameters for the run.
settings = openmc.Settings()
settings.source = source
settings.batches = 20
settings.inactive = 5
settings.particles = 10000
settings.export_to_xml()

#This should likely be set to the anount of available threads.
#settings.max_particles_in_flight=12
openmc.run()